<a href="https://colab.research.google.com/github/ncourtel1/strava-urban-mobility-pipeline/blob/main/notebook/01_pipeline_strava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install stravalib pandas polyline requests folium python-dotenv stravaio wearipedia

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
from stravalib import Client
import datetime
import polyline
import urllib
import requests
import json
from stravaio import StravaIO, strava_oauth2
import folium
import logging
import wearipedia

  return datetime.utcnow().replace(tzinfo=utc)

  from scipy.ndimage.filters import uniform_filter1d



## 2. Authentication and Authorization
To obtain access to data, authorization is required. Put in your client id, client secret token and refresh token for your Strava account. We'll use this to extract the data in the sections below.

In [7]:
from dotenv import load_dotenv
import os

# Charger le fichier .env
load_dotenv('.env')

# Récupérer les variables
client_id = os.getenv('STRAVA_CLIENT_ID')
client_secret = os.getenv('STRAVA_CLIENT_SECRET')

In [8]:
params_oauth = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": f"http://localhost:8000/authorization_successful",
    "scope": "read,profile:read_all,activity:read",
    "approval_prompt": "force"
}
values_url = urllib.parse.urlencode(params_oauth)
base_url = 'https://www.strava.com/oauth/authorize'
authorize_url = base_url + '?' + values_url
print(authorize_url)

https://www.strava.com/oauth/authorize?client_id=181205&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fauthorization_successful&scope=read%2Cprofile%3Aread_all%2Cactivity%3Aread&approval_prompt=force


In [9]:
AUTHORIZATION_CODE = "778cf6f8f623bb95ccc562f1133c7255098344ae" #@param {type: "string"}

In [10]:
def refresh_strava_access_token(client_id, client_secret, AUTHORIZATION_CODE):
    """
    Refresh the Strava access token using the refresh token.

    Args:
        client_id (str): Your Strava API client ID
        client_secret (str): Your Strava API client secret
        refresh_token (str): The refresh token

    Returns:
        str: The new access token
    """
    url = "https://www.strava.com/api/v3/oauth/token"
    payload = {
        'client_id': client_id,
        'client_secret': client_secret,
        'code': AUTHORIZATION_CODE,
        'grant_type': 'authorization_code'
    }

    try:
        response = requests.post(url, data=payload)
        response.raise_for_status()
        data = response.json()

        new_access_token = data.get('access_token')
        new_refresh_token = data.get('refresh_token') # Get the new refresh token as well

        if not new_access_token:
            print("Warning: No new access token found in response")

        if not new_refresh_token:
            print("Warning: No new refresh token found in response")

        global refresh_token
        refresh_token = new_refresh_token

        return new_access_token

    except requests.exceptions.RequestException as e:
        print(f"Error making request to refresh token: {e}")
        return None

# Refresh the access token
global access_token
access_token = refresh_strava_access_token(client_id, client_secret, AUTHORIZATION_CODE)

if access_token:
    print("New access token obtained.")
    print("New access token:", access_token)
    print("New refresh token:", refresh_token)
else:
    print("Failed to obtain a new access token.")

New access token obtained.
New access token: 8945d8f42c966d72f114aa43d244379d6b932ff7
New refresh token: 91bf1b3654e117efc7b7bfb3d4fc9c97fd174b26


## 3. Data Extraction

### En utilisant Stravalib

In [38]:
from stravalib.client import Client
import pandas as pd
import time

client = Client(access_token=access_token)
client.refresh_token = refresh_token

activities = client.get_activities(limit=50)
segments_data = []

for activity in activities:
    try:
        detailed_activity = client.get_activity(activity.id)

        if hasattr(detailed_activity, "segment_efforts") and detailed_activity.segment_efforts:
            for effort in detailed_activity.segment_efforts:
                seg = effort.segment

                # conversion sécurisée pour les floats
                distance = None
                try:
                    distance = float(seg.distance)
                except Exception:
                    distance = None

                segments_data.append({
                    "activity_id": activity.id,
                    "activity_name": activity.name,
                    "activity_type": activity.type,
                    "segment_id": getattr(seg, "id", None),
                    "segment_name": getattr(seg, "name", None),
                    "distance_m": distance,
                    "avg_grade": getattr(seg, "average_grade", None),
                    "elev_high": getattr(seg, "elevation_high", None),
                    "elev_low": getattr(seg, "elevation_low", None),
                    "start_lat": getattr(seg.start_latlng, "lat", None) if seg.start_latlng else None,
                    "start_lng": getattr(seg.start_latlng, "lon", None) if seg.start_latlng else None,
                    "end_lat": getattr(seg.end_latlng, "lat", None) if seg.end_latlng else None,
                    "end_lng": getattr(seg.end_latlng, "lon", None) if seg.end_latlng else None,
                    "starred": getattr(seg, "starred", None),
                })

        time.sleep(1)  # limite API

    except Exception as e:
        print(f"Erreur sur l’activité {activity.id}: {e}")
        continue

df_segments = pd.DataFrame(segments_data)
print(f"{len(df_segments)} segments extraits.")
df_segments.head()


Erreur sur l’activité 16188685770: 597 Server Error: Unknown [None]
Erreur sur l’activité 16169083327: 597 Server Error: Unknown [None]


KeyboardInterrupt: 

In [22]:
df_segments.to_csv("strava_segments.csv", index=False)
df_segments.to_excel("strava_segments.xlsx", index=False)

### En utilisant wearipedia

In [11]:
#set start and end dates
start_date='2021-09-01' #@param {type:"string"}
end_date='2025-10-31' #@param {type:"string"}
synthetic = False #@param {type:"boolean"}

In [12]:
device = wearipedia.get_device("strava/strava")

if not synthetic:
    device.authenticate({
    'client_id':client_id,
    'client_secret':client_secret,
    'refresh_token':refresh_token
    })

params = {"start_date": start_date, "end_date": end_date}

distance = device.get_data("distance", params=params)
moving_time = device.get_data("moving_time", params=params)
elapsed_time = device.get_data("elapsed_time", params=params)
total_elevation_gain = device.get_data("total_elevation_gain", params=params)
average_speed = device.get_data("average_speed", params=params)
max_speed = device.get_data("max_speed", params=params)
average_heartrate = device.get_data("average_heartrate", params=params)
max_heartrate = device.get_data("max_heartrate", params=params)
map_summary_polyline = device.get_data("map_summary_polyline", params=params)
elev_high = device.get_data("elev_high", params=params)
elev_low = device.get_data("elev_low", params=params)
average_cadence = device.get_data("average_cadence", params=params)
average_watts = device.get_data("average_watts", params=params)
kilojoules = device.get_data("kilojoules", params=params)


  return datetime.utcnow().replace(tzinfo=utc)



Requesting Token...

Access Token = 8945d8f42c966d72f114aa43d244379d6b932ff7



  return datetime.utcnow().replace(tzinfo=utc)



## 4. Data Exporting

Exporting to CSV/XLSX

We will thus export steps, heart rates, and breath rates all as separate files.

In [13]:
complete = {
    "distance": distance,
    'moving_time':moving_time,
    'elapsed_time':elapsed_time,
    'total_elevation_gain':total_elevation_gain,
    'average_speed':average_speed,
    'max_speed':max_speed,
    'average_heartrate':average_heartrate,
    'max_heartrate':max_heartrate,
    'map_summary_polyline':map_summary_polyline,
    'elev_high':elev_high,
    'elev_low':elev_low,
    'average_cadence':average_cadence,
    'average_watts':average_watts,
    'kilojoules':kilojoules
}

In [14]:
def kmh_to_pace_float(kmh):
    """
    Convertit une vitesse (km/h) en allure (min/km) au format float M.SS.
    Exemple : 8.9 km/h -> 6.44 (soit 6 min 44 s / km)
    """
    if kmh <= 0:
        return None

    # Allure en minutes/km
    pace = 60 / kmh
    minutes = int(pace)
    seconds = (pace - minutes) * 60

    # exemple : 6 min + 44 sec → 6 + 44/100 = 6.44 (float)
    pace_float = minutes + (seconds / 100)

    # arrondi propre
    return round(pace_float, 2)


# average_speed_df = pd.DataFrame.from_dict(complete['average_speed'])
average_speed_df = pd.DataFrame(average_speed)
average_speed_df['average_speed_km_per_hour'] = round(average_speed_df['average_speed'] * 3.6, 2)
average_speed_df['average_speed_min_per_km'] = average_speed_df['average_speed_km_per_hour'].apply(kmh_to_pace_float)
average_speed_df.to_csv('average_speed.csv')
average_speed_df.to_excel('average_speed.xlsx')

# Transformer la liste de dictionnaires en DataFrame
max_speed_df = pd.DataFrame(max_speed)

# Convertir les vitesses de m/s → km/h
max_speed_df["max_speed_km_per_hour"] = round(max_speed_df["max_speed"] * 3.6, 2)
max_speed_df["max_speed_min_per_km"] = max_speed_df["max_speed_km_per_hour"].apply(kmh_to_pace_float)

# Sauvegarde en CSV et Excel
max_speed_df.to_csv("max_speed.csv")
max_speed_df.to_excel("max_speed.xlsx")

  return datetime.utcnow().replace(tzinfo=utc)



In [19]:
# distance_df = pd.DataFrame.from_dict(complete['distance'])
# distance_df.to_csv('data/distance.csv')
# distance_df.to_excel('data/distance.xlsx')

# moving_time_df = pd.DataFrame.from_dict(complete['moving_time'])
# moving_time_df.to_csv('data/moving_time.csv')
# moving_time_df.to_excel('data/moving_time.xlsx')

# elapsed_time_df = pd.DataFrame.from_dict(complete['elapsed_time'])
# elapsed_time_df.to_csv('data/elapsed_time.csv')
# elapsed_time_df.to_excel('data/elapsed_time.xlsx')

total_elevation_gain_df = pd.DataFrame(total_elevation_gain)
total_elevation_gain_df['total_elevation_gain_in_meters'] = total_elevation_gain_df['total_elevation_gain']
total_elevation_gain_df = total_elevation_gain_df.drop(columns=['total_elevation_gain'])
total_elevation_gain_df.to_csv('total_elevation_gain.csv')
total_elevation_gain_df.to_excel('total_elevation_gain.xlsx')

# average_heartrate_df = pd.DataFrame.from_dict(complete['average_heartrate'])
# average_heartrate_df.to_csv('data/average_heartrate.csv')
# average_heartrate_df.to_excel('data/average_heartrate.xlsx')

# max_heartrate_df = pd.DataFrame.from_dict(complete['max_heartrate'])
# max_heartrate_df.to_csv('data/max_heartrate.csv')
# max_heartrate_df.to_excel('data/max_heartrate.xlsx')

# map_summary_polyline_df = pd.DataFrame.from_dict(complete['map_summary_polyline'])
# map_summary_polyline_df.to_csv('data/map_summary_polyline.csv')
# map_summary_polyline_df.to_excel('data/map_summary_polyline.xlsx')

elev_high_df = pd.DataFrame(elev_high)
elev_high_df['elev_high_in_meters'] = elev_high_df['elev_high']
elev_high_df = elev_high_df.drop(columns=['elev_high'])
elev_high_df.to_csv('elev_high_in_meters.csv')
elev_high_df.to_excel('elev_high_in_meters.xlsx')
# elev_high_df = pd.DataFrame.from_dict(complete['elev_high_in_meters'])
# elev_high_df.to_csv('elev_high_in_meters.csv')
# elev_high_df.to_excel('elev_high_in_meters.xlsx')

# elev_low_df = pd.DataFrame.from_dict(complete['elev_low'])
# elev_low_df.to_csv('data/elev_low.csv')
# elev_low_df.to_excel('data/elev_low.xlsx')

# average_cadence_df = pd.DataFrame.from_dict(complete['average_cadence'])
# average_cadence_df.to_csv('data/average_cadence.csv')
# average_cadence_df.to_excel('data/average_cadence.xlsx')

# average_watts_df = pd.DataFrame.from_dict(complete['average_watts'])
# average_watts_df.to_csv('data/average_watts.csv')
# average_watts_df.to_excel('data/average_watts.xlsx')

# kilojoules_df = pd.DataFrame.from_dict(complete['kilojoules'])
# kilojoules_df.to_csv('data/kilojoules.csv')
# kilojoules_df.to_excel('data/kilojoules.xlsx')

# complete_df = pd.DataFrame.from_dict(complete)
# complete_df.to_csv('data/complete.csv')
# complete_df.to_excel('data/complete.xlsx')


  return datetime.utcnow().replace(tzinfo=utc)



In [ ]:
!zip -r data.zip data/

  adding: data/ (stored 0%)
  adding: data/distance.xlsx (deflated 6%)
  adding: data/map_summary_polyline.csv (deflated 32%)
  adding: data/total_elevation_gain.csv (deflated 61%)
  adding: data/elev_high.csv (deflated 60%)
  adding: data/strava_segments.csv (deflated 88%)
  adding: data/complete.csv (deflated 73%)
  adding: data/max_heartrate.xlsx (deflated 7%)
  adding: data/average_speed.csv (deflated 60%)
  adding: data/moving_time.csv (deflated 59%)
  adding: data/elapsed_time.xlsx (deflated 6%)
  adding: data/elapsed_time.csv (deflated 59%)
  adding: data/map_summary_polyline.xlsx (deflated 1%)
  adding: data/average_heartrate.xlsx (deflated 7%)
  adding: data/total_elevation_gain.xlsx (deflated 7%)
  adding: data/average_speed.xlsx (deflated 6%)
  adding: data/moving_time.xlsx (deflated 6%)
  adding: data/kilojoules.csv (deflated 59%)
  adding: data/average_cadence.csv (deflated 60%)
  adding: data/kilojoules.xlsx (deflated 7%)
  adding: data/strava_segments.xlsx (deflated 4%)


## Visualization

### Activities

In [31]:
client_io = StravaIO(access_token)
athlete = client.get_logged_in_athlete()
activities = client.get_logged_in_athlete_activities(after=20170101)

Fetched 30, the latests is on 2025-05-07 13:32:10+00:00
Fetched 30, the latests is on 2025-08-09 10:43:33+00:00
Fetched 28, the latests is on 2025-10-19 08:49:04+00:00


In [56]:
m = folium.Map(
    tiles="cartodbpositron",
    location=[49.4435, 1.0984],
    zoom_start=11.5,
    control_scale=True
)
folium.TileLayer("cartodbpositron").add_to(m)
folium.TileLayer("cartodbdark_matter").add_to(m)
folium.LayerControl().add_to(m)

In [33]:
def downsample(l, n):
    """Returns every nth element from list l. Returns the
    original list if n is set to 1.
    Used to reduce the number of GPS points per activity,
    to improve performance of the website.
    """

    return l[0::n]

def map_activities(activities, folium_map, opacity, weight):
    if len(activities) == 0:
        logging.info("No activities found, returning empty folium map.")
        return folium_map

    for a in activities:
        if a.type == "Workout":
            continue
        streams = client_io.get_activity_streams(a.id, athlete.id)
        try:
            points = list(zip(streams.lat, streams.lng))
            points = downsample(l=points, n=2)
            if a.type == "Run":
                folium.PolyLine(
                    locations=points, color="#ff9933", opacity=opacity, weight=weight
                ).add_to(folium_map)
            else:
                folium.PolyLine(
                    locations=points, color="#cc00ff", opacity=opacity, weight=weight
                ).add_to(folium_map)
            logging.critical("Mapped activity with id: {}".format(a.id))
        except Exception:
            logging.error("Could not map activity with id: {}".format(a.id))

    return folium_map

In [34]:
m = map_activities(
    activities=activities,
    folium_map=m,
    opacity=0.5,
    weight=1
)

CRITICAL:root:Mapped activity with id: 13901459544
CRITICAL:root:Mapped activity with id: 13906587182
CRITICAL:root:Mapped activity with id: 13927346809
CRITICAL:root:Mapped activity with id: 13943708973
CRITICAL:root:Mapped activity with id: 13954304186
CRITICAL:root:Mapped activity with id: 13972077955
CRITICAL:root:Mapped activity with id: 13993907029
CRITICAL:root:Mapped activity with id: 14002587041
CRITICAL:root:Mapped activity with id: 14020823219
CRITICAL:root:Mapped activity with id: 14038383362
CRITICAL:root:Mapped activity with id: 14061678650
CRITICAL:root:Mapped activity with id: 14068494201
CRITICAL:root:Mapped activity with id: 14078305148
CRITICAL:root:Mapped activity with id: 14088066118
CRITICAL:root:Mapped activity with id: 14106868843
CRITICAL:root:Mapped activity with id: 14147587813
CRITICAL:root:Mapped activity with id: 14164796198
CRITICAL:root:Mapped activity with id: 14175557185
CRITICAL:root:Mapped activity with id: 14194133150
CRITICAL:root:Mapped activity w

In [35]:
m

### Segments

In [57]:
from tqdm import tqdm
import logging
import folium
import polyline

def map_segments(segments_df, folium_map, opacity=0.6, weight=3, limit=10):
    """
    Affiche les segments Strava sur une carte Folium.
    """
    if len(segments_df) == 0:
        logging.warning("Aucun segment à afficher.")
        return folium_map

    batch = segments_df.iloc[:limit]

    for _, seg in tqdm(batch.iterrows(), total=len(batch), desc="Mapping segments"):
        try:
            seg_id = seg.get("segment_id")
            if seg_id is None:
                logging.warning("Segment sans ID, skipped")
                continue

            # Récupération des infos détaillées via StravaIO
            segment_details = client.get_segment(seg_id)

            # Décodage du polyline si disponible
            poly = getattr(segment_details.map, "polyline", None)
            if not poly:
                logging.warning(f"Aucun polyline pour le segment {seg_id}")
                continue

            coords = polyline.decode(poly)

            # Lecture des colonnes correctes
            name = seg.get("segment_name", "Segment inconnu")
            distance = seg.get("distance_m", 0) / 1000  # km
            elev_high = seg.get("elev_high", 0)
            elev_low = seg.get("elev_low", 0)
            avg_grade = seg.get("avg_grade", 0)

            gain = elev_high - elev_low

            # Code couleur selon le dénivelé
            if gain < 20:
                color = "#00cc44"  # vert
            elif gain < 50:
                color = "#ffa500"  # orange
            else:
                color = "#ff0000"  # rouge

            # Tracé sur la carte
            folium.PolyLine(
                locations=coords,
                color=color,
                opacity=opacity,
                weight=weight,
                popup=folium.Popup(
                    f"<b>{name}</b><br>Distance : {distance:.2f} km<br>Dénivelé : {gain:.1f} m<br>Pente : {avg_grade:.1f}%",
                    max_width=250
                )
            ).add_to(folium_map)

        except Exception as e:
            logging.error(f"Erreur sur le segment {seg_id} : {e}")

    return folium_map


In [50]:
print(df_segments.head())

  return datetime.utcnow().replace(tzinfo=utc)



   activity_id          activity_name activity_type  segment_id  \
0  16169083327  Prépa Trail 42km W2S3    root='Run'    27925332   
1  16169083327  Prépa Trail 42km W2S3    root='Run'    36062539   
2  16169083327  Prépa Trail 42km W2S3    root='Run'    23635816   
3  16169083327  Prépa Trail 42km W2S3    root='Run'    30599310   
4  16169083327  Prépa Trail 42km W2S3    root='Run'    29100404   

                        segment_name  distance_m  avg_grade  elev_high  \
0                        Libération        205.2       11.5      113.2   
1              Faux plat boulangerie       307.4        0.9      151.0   
2              Les 70mD+ de Francis        477.0       14.2      136.6   
3  Rue d'etancourt + rue gibert down       712.7       -8.3      -38.0   
4              Rampe St hilaire down       562.6       -4.8       48.5   

   elev_low  start_lat  start_lng    end_lat   end_lng starred  
0      89.5  49.452425   1.109206  49.453682  1.111214    None  
1     148.1  49.456602

In [58]:
m = map_segments(df_segments, m, opacity=0.6, weight=3, limit=20)

Mapping segments: 100%|██████████| 20/20 [00:17<00:00,  1.16it/s]


In [59]:
m